In [ ]:
import math
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
def f(x):
    return 3*x**2-4*x+5

In [ ]:
f(3.0)

In [ ]:
xs=np.arange(-5,5,0.25)#从-5到5(不含)步长0.25
ys=f(xs)
plt.plot(xs,ys);

In [ ]:
h=0.000001
x=-3
(f(x+h)-f(x))/h

In [ ]:
h=0.0001

a=2.0
b=-3.0
c=10.0

d1=a*b+c
b+=h
d2=a*b+c

print('d1',d1)
print('d2',d2)
print('slope',(d1-d2)/h)

In [ ]:
class Value:

    def __init__(self,data,_children=(),_op='',label=''):#初始化class,children是一个空tuple
        self.data=data
        self.grad=0.0
        self._backward=lambda:None#默认是空函数,不做任何操作
        self._prev=set(_children)#把元组转换为集合
        self._op=_op
        self.label=label

    def __repr__(self):
        return f"Value(data={self.data})"#wrapper函数提供一个更加美观的打印结果
    
    def __add__(self,other):
        other=other if isinstance(other,Value) else Value(other)#若other是Value的实例直接返回，若不是则将其包装为Value类型
        out=Value(self.data+other.data,(self,other),'+')

        def _backward():
            self.grad+=1.0*out.grad
            other.grad+=1.0*out.grad
        out._backward=_backward

        return out
    
    def __neg__(self):#-self
        return self*-1
    
    def __sub__(self,other):#self-other
        return self+(-other)
    
    def __mul__(self,other):
        other=other if isinstance(other,Value) else Value(other)
        out=Value(self.data*other.data,(self,other),'*')

        def _backward():
            self.grad+=other.data*out.grad
            other.grad+=self.data*out.grad
        out._backward=_backward

        return out
    
    def __pow__(self,other):
        assert isinstance(other,(float,int))#现在仅支持整数型或浮点型
        out=Value(self.data**other,(self,),f'**{other}')

        def _backward():
            self.grad+=other*self.data**(other-1)*out.grad#other就是数字，不需要.data
        out._backward=_backward

        return out
    
    def tanh(self):
        x=self.data
        t=(math.exp(2*x)-1)/(math.exp(2*x)+1)
        out=Value(t,(self,),'tanh')

        def _backward():
            self.grad+=(1-t**2) * out.grad
        out._backward=_backward

        return out
    
    def __rmul__(self,other):# other * self
        return self*other
    
    def __truediv__(self,other):#self / other
        return self*other**-1
    
    def exp(self):
        x=self.data
        out=Value(math.exp(x),(self,),'exp')

        def _backward():
            self.grad+=out.data*out.grad
        out._backward=_backward
        
        return out
    
    def backward(self):
        #拓扑排序topological sort
        topo=[]
        visited=set()#避免重复遍历同一个节点
        def build_topo(v):
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)#visited 集合保证了 topo 里没有重复节点
        build_topo(self)

        self.grad=1.0#若不初始化为1，grad仍旧保持0则反向传播的时候只要相乘前面的grad就全部为零了
        for node in reversed(topo):
            node._backward()
    


In [ ]:
a=Value(2.0,label='a')
a.exp()
b=Value(-3.0,label='b')
a**b.data

# c=Value(10.0,label='c')
# e=a*b; e.label='e'
# d=e+c;d.label='d'
# f=Value(-2.0,label='f')
# L=f*d;L.label='L'
#(a.__mul__(b)).__add__(c)

In [ ]:
from graphviz import Digraph #画计算图

def trace(root):
    #构建计算图中的所有边和节点
    nodes,edges=set(),set()
    def build(v):
        if v not in nodes:
            nodes.add(v)
            for child in v._prev:
                edges.add((child,v))
                build(child)
    build(root)
    return nodes,edges

def draw_dot(root):
    dot=Digraph(format='svg',graph_attr={'rankdir':'LR'})#LR:left to right
    nodes,edges=trace(root)
    for n in nodes:
        uid=str(id(n))
        dot.node(name=uid,label="{%s|data %.4f|grad %.4f}"%(n.label,n.data,n.grad),shape='record')
        if n._op:
            dot.node(name=uid+n._op,label=n._op)# 创建操作节点
            dot.edge(uid+n._op,uid)#操作节点 → 数值节点

    for n1,n2 in edges:
        dot.edge(str(id(n1)),str(id(n2))+n2._op)# 输入节点→操作节点

    return dot

In [ ]:
#inputs x1,x2
x1=Value(2.0,label='x1')
x2=Value(0.0,label='x2')
#weights w1,w2
w1=Value(-3.0,label='w1')
w2=Value(1.0,label='w2')
#bias of the neuron
b=Value(6.8813735870195432,label='b')
#x1*w1 +x2*w2 + b
x1w1=x1*w1;x1w1.label='x1*w1'
x2w2=x2*w2;x2w2.label='x2*w2'
x1w1x2w2=x1w1+x2w2;x1w1x2w2.label='x1*w1 + x2*w2'
n=x1w1x2w2+b;n.label='n'
#---
e=(2*n).exp();e.label='e'
o=(e-1)/(e+1)
#---
o.label='o'
o.backward()

In [ ]:
draw_dot(o)

In [ ]:
import torch

x1=torch.tensor([2.0]).double();x1.requires_grad=True#由于是叶节点默认不计算grad，这里专门都设置为需要计算
x2=torch.tensor([0.0]).double();x2.requires_grad=True
w1=torch.tensor([-3.0]).double();w1.requires_grad=True
w2=torch.tensor([1.0]).double();w2.requires_grad=True
b=torch.tensor([6.8813735870195432]).double();b.requires_grad=True
n=x1*w1+x2*w2+b
o=torch.tanh(n)

print(o.item())#去除张量结构
o.backward()

print("---")
print('x1',x1.grad.item())
print('w1',w1.grad.item())
print('x2',x2.grad)
print('w2',w2.grad)

In [ ]:
torch.Tensor([2.0]).double().dtype #PyTorch默认float32

In [ ]:
class Neuron:#define a Neuron
    def __init__(self,nin):#nin是神经元接收的输入数量
        self.w=[Value(np.random.uniform(-1,1)) for _ in range(nin)]#创建数组，大小为nin数值在-1到1间
        self.b=Value(np.random.uniform(-1,1))

    def __call__(self,x):
        # w * x + b
        act=sum((wi*xi for wi,xi in zip(self.w,x)),self.b)#get an activation得到激活值,初始值为self.b
        out=act.tanh()
        return out
    
class Layer:#每层含有多个神经元
    def __init__(self,nin, nout):
        self.neurons=[Neuron(nin) for _ in range(nout)] 

    def __call__(self,x):
        outs=[n(x) for n in self.neurons]
        return outs
    
x=[2.0,3.0]
n=Neuron(2)# 2. 创建神经元（2个输入）n = Neuron(2)
n(x)# ↓ 等价于n.__call__(x)